In [1]:
import pandas as pd
import numpy as np
import os
import nltk
import random

In [ ]:
import itertools
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
# from keras.callbacks import EarlyStopping

from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer

In [18]:
## Load data from CSV file to dataframe

# df_sentiment = pd.read_csv("./sentiment_gop_cleaned.csv", engine='python')
df_sentiment = pd.read_csv("./cleaned_airline_sentiment.csv", engine='python')
# df_sentiment.drop(['Unnamed: 0'], axis=1, inplace=True)
# print(df_gop.shape)
df_sentiment.head()

,tweet_id,tweet_created,sentiment,full_text,cleaned_text
0,570301130888122368,2015-02-24 11:15:59 -0800,non-negative,@VirginAmerica plus you've added commercials t...,plus you have added commercials to the experie...
1,570301083672813571,2015-02-24 11:15:48 -0800,non-negative,@VirginAmerica I didn't today... Must mean I n...,i did not today must mean i need to take anoth...
2,570301031407624196,2015-02-24 11:15:36 -0800,negative,@VirginAmerica it's really aggressive to blast...,it is really aggressive to blast obnoxious ent...
3,570300817074462722,2015-02-24 11:14:45 -0800,negative,@VirginAmerica and it's a really big bad thing...,and it is a really big bad thing about it
4,570300767074181121,2015-02-24 11:14:33 -0800,negative,@VirginAmerica seriously would pay $30 a fligh...,seriously would pay $30 a flight for seats tha...


In [3]:
df_sentiment['sentiment'].value_counts()

negative        8822
non-negative    4402
Name: sentiment, dtype: int64

In [4]:
# df_sentiment = df_sentiment.sort_values(by=['sentiment'])
# df_sentiment.reset_index(drop=True)
# df_sentiment= df_sentiment.iloc[3000:]
# df_sentiment = df_sentiment.sample(frac=1).reset_index(drop=True)
# df_sentiment['sentiment'].value_counts()

In [5]:
# df_sentiment['sentiment'] = pd.Categorical(df_sentiment['sentiment'], ["Neutral", "Negative", "Positive"])
# df_sentiment = df_sentiment.sort_values(by=['sentiment'])
# df_sentiment.reset_index(drop=True)
# df_sentiment= df_sentiment.iloc[700:]
# df_sentiment = df_sentiment.sample(frac=1).reset_index(drop=True)
# df_sentiment['sentiment'].value_counts()
def change_vals(row):
    if row == 'Neutral' or row == 'Positive':
        return "Non-Negative"
    else:
        return row
df_sentiment['sentiment'] = df_sentiment['sentiment'].apply(change_vals)

In [6]:
data = df_sentiment
data['sentiment'].value_counts()

negative        8822
non-negative    4402
Name: sentiment, dtype: int64

In [8]:
max_len = 130
num_labels = 2
vocab_size = 4000
batch_size = 32
num_epochs = 10
embed_dim = 128
lstm_out = 196

tokenizer = Tokenizer(num_words=vocab_size, split=' ')
tokenizer.fit_on_texts(data['cleaned_text'].values)
X = tokenizer.texts_to_sequences(data['cleaned_text'].values)
X = pad_sequences(X)
Y = pd.get_dummies(data['sentiment']).values

print(len(data['sentiment'].values))
arr_y = np.array(Y)

sanity_check = {'text': data['cleaned_text'].values, 'sentiment': data['sentiment'].values, 'negative':arr_y[:,0], 'non-negative': arr_y[:,1]}
sanity_check = pd.DataFrame(data=sanity_check)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

encoder = LabelBinarizer()
encoder.fit(Y)

13224


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [9]:
sanity_check.sample(50)

,text,sentiment,negative,non-negative
968,look at this beauty :winking_face: dc 10 unite...,non-negative,0,1
4865,you can do better than having someone on hold...,negative,1,0
4553,just realized i had the wrong departure date t...,non-negative,0,1
9410,i was told a return call by a human,negative,1,0
8102,i know this is probably a no but is there a wa...,non-negative,0,1
7993,well i did miss it but gate agents had rebooke...,non-negative,0,1
11834,terrible response how about someone picking up...,negative,1,0
12210,husband i complain about same flight he gets ...,negative,1,0
1122,left item n the seatback on ua1260 is there an...,non-negative,0,1
5162,you could be a little more accommodating with ...,negative,1,0


In [10]:
X_train.shape, Y_train.shape

((8860, 89), (8860, 2))

In [11]:
print(sum(Y_test))

[130 138]


In [12]:
# #let us build a basic model
# model = Sequential()
# model.add(Dense(512, input_shape=(vocab_size,)))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.3))
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))
# model.summary()

# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

model = Sequential()
model.add(Embedding(vocab_size, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_labels,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 89, 128)           512000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 89, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 767,194
Trainable params: 767,194
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# num_epochs = 10
# batch_size = 128
# history = model.fit(X_train, Y_train,
#                     batch_size=batch_size,
#                     epochs=num_epochs,
#                     verbose=2,
#                     validation_split=0.2)

model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size, verbose=2)

/Users/JamesHowe/miniconda3/envs/ai_env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
 - 64s - loss: 0.4705 - accuracy: 0.7766
Epoch 2/10
 - 55s - loss: 0.3292 - accuracy: 0.8594
Epoch 3/10
 - 57s - loss: 0.2690 - accuracy: 0.8880
Epoch 4/10
 - 55s - loss: 0.2325 - accuracy: 0.9033
Epoch 5/10
 - 54s - loss: 0.1897 - accuracy: 0.9227
Epoch 6/10
 - 53s - loss: 0.1656 - accuracy: 0.9330
Epoch 7/10
 - 55s - loss: 0.1455 - accuracy: 0.9432
Epoch 8/10
 - 51s - loss: 0.1235 - accuracy: 0.9519
Epoch 9/10
 - 49s - loss: 0.1034 - accuracy: 0.9614
Epoch 10/10
 - 56s - loss: 0.0950 - accuracy: 0.9635


In [119]:
score, acc = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=2)

print('Classification Accuracy on Test Set:', acc * 100, '%')

Classification Accuracy on Test Set: 81.59945011138916 %


In [98]:
text_labels = encoder.classes_

res_dict = {'predicted': [] }
for i in range(len(X_test)):
    prediction = model.predict(np.array([X_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
    res_dict['predicted'].append(predicted_label)

df_res = pd.DataFrame(data=res_dict)
df_res['predictedLabel'] = ["Non-Negative" if y == 1 else "Negative" for y in df_res['predicted']]
df_res['actual'] = ["Non-Negative" if y.tostring() == np.asarray([0, 1], dtype=np.uint8).tostring() else "Negative" for y in Y_test]

print(df_res['predicted'].value_counts())
print(df_res['actual'].value_counts())

0    2885
1    1479
Name: predicted, dtype: int64
Negative        2946
Non-Negative    1418
Name: actual, dtype: int64


In [99]:
## Reverse the tokenized vectors to re-generate the cleaned versions of tweets

reverseTokenizerDict = dict(map(reversed, tokenizer.word_index.items()))
predTweets = []
for i in X_test:
    s = ""
    for j in i:
        if j != 0:
            s += reverseTokenizerDict[j] + " "
    predTweets.append(s)
        
df_res['generatedCleanTweets'] = predTweets

(4364, 89) (4364, 2)


In [118]:
pd.options.display.max_colwidth = 100
df_res.head(25)

,predicted,predictedLabel,actual,generatedCleanTweets
0,1,Non-Negative,Non-Negative,thank you excited to be working with you guys
1,0,Negative,Negative,thank you for letting me luggage is still in denver but i am in phl neveragain disappointed
2,0,Negative,Negative,for our delays i am out of more money because of you
3,1,Non-Negative,Non-Negative,so the fares i see for flights in fall are the lowest they will be
4,0,Negative,Negative,understand weather is an issue but on time cancelled flighted reinstated cancelled flighted with...
5,0,Negative,Negative,why can not i find a cheap flight from dc to st louis the prices went up like crazy for april
6,0,Negative,Negative,service today missed my connecting flight then the customer service desk was terrible to me real...
7,0,Negative,Negative,status shows delayed it was just waiting for takeoff so did it depart it certainly did not at
8,0,Negative,Negative,i booked it on us airways site do not see a cancelled flight link
9,0,Negative,Negative,what is happening with the flight from fll to sfo why the delay and the reroute


In [117]:
## Find accuracy of predictions on class-by-class basis
negs = df_res.loc[df_res['actual'] == "Negative"]
nons = df_res.loc[df_res['actual'] == "Non-Negative"]

accNegs = sum(np.where(negs['predictedLabel'] == "Negative", 1, 0)) / negs.shape[0] * 100
accNons = sum(np.where(nons['predictedLabel'] == "Non-Negative", 1, 0)) / nons.shape[0] * 100

print("Classification Accuracy on Negative Data: ", accNegs, "%")
print("Classification Accuracy on Non-Negative Data: ", accNons, "%")


Classification Accuracy on Negative Data:  85.33604887983707 %
Classification Accuracy on Non-Negative Data:  73.83638928067701 %


## Clustering

In [109]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans


model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)

13224


In [110]:
print(X[0])
print(X[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  528    4   15
 1160 2391    1    3  181]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   2  71   9  89 722 549   2  66   1 138 136 175]
